In [1]:
from traj.traj_pre_3d import xyz_traj_handle
import numpy as np
from tqdm.auto import tqdm
import networkx as nx
from sklearn.neighbors import KDTree

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def find_pairs_kdtree(tracearr, min_distance, max_distance):
    coords = tracearr[:, 2:5]
    
    tree = KDTree(coords)
    if min_distance > 0:
        
        pairs_max = tree.query_radius(coords, max_distance)
        
        pairs_min = tree.query_radius(coords, min_distance)
        
        unique_pairs = set()
        for i, neighbors in enumerate(pairs_max):
            for j in neighbors:
                if j > i and j not in pairs_min[i]:
                    unique_pairs.add((i, j))
    else:
        pairs = tree.query_radius(coords, max_distance)
    
        unique_pairs = set()
        for i, neighbors in enumerate(pairs):
            for j in neighbors:
                if j > i:
                    unique_pairs.add((i, j))
    return np.array(list(unique_pairs))
class timestep_detect:
    def __init__(self, script_path):
        self.lammps_script_path = script_path

    def found_dump(self, process_type = "eq"):
        with open(self.lammps_script_path, "r") as f:
            script_document = f.readlines()
        if process_type == "eq":
            for line in script_document:
                words = line.split()
                if len(words) > 2:
                    if words[1] == "dumpfreq":
                        self.timestep = int(words[-1][:-1])
                        break
            print(f"Info: found timestep = {self.timestep}")
        elif process_type == "deform":
            for line in script_document:
                words = line.split()
                if len(words) > 2:
                    if words[1] == "dumpfreq":
                        self.timestep = int(words[-1][:-1])
                        break
            print(f"Info: found timestep = {self.timestep}")
        else:
            print(f"Error: invalid key words: {process_type}")

class Stress_detect:
    def __init__(self, stress_path):
        self.stress_path = stress_path

    def found_stress(self, total_frames):
        with open(self.stress_path, "r") as f:
            stress_document = f.readlines()
        stress = []
        for line in stress_document[2:]:
            words = line.split()
            stress.append(float(words[1]))
        stress = np.array(stress)
        frame = stress.shape[0]//(total_frames-1)
        stress_framed = stress[:len(stress):frame]
        stress_framed = np.append(stress_framed, stress[-1])
        return stress_framed

In [ ]:
class network_analysis_function:
    @staticmethod
    def count_subgraphs(G):
        count = 0
        max_connected_subgraph_size = 0
        total_nodes_in_subgraphs = 0  
    
        for component in nx.connected_components(G):
            subgraph = G.subgraph(component)
    
            if subgraph.number_of_nodes() < 2:
                continue
    
            degree_dict = dict(subgraph.degree())
            nodes_with_degree_ge_3 = [node for node, degree in degree_dict.items() if degree >= 3]
    
            if len(nodes_with_degree_ge_3) < 2:
                continue
    
            count += 1
    
            max_connected_subgraph_size = max(max_connected_subgraph_size, subgraph.number_of_nodes())
    
            total_nodes_in_subgraphs += subgraph.number_of_nodes()
    
        ratio_total = total_nodes_in_subgraphs / G.number_of_nodes()
        ratio_max = max_connected_subgraph_size / G.number_of_nodes()
    
        return count, max_connected_subgraph_size, ratio_total, ratio_max

    def cal_average_degree(G):
        degrees = dict(G.degree())
        filtered_degrees = {k: v for k, v in degrees.items() if v >= 2}
        total_degrees = sum(filtered_degrees.values())
        num_nodes = len(filtered_degrees)
        if num_nodes == 0:
            return 0
        return total_degrees / num_nodes

    def cal_average_betweenness_centrality(G):
        betweenness_centrality = nx.betweenness_centrality(G)
        average_betweenness_centrality = sum(betweenness_centrality.values()) / len(betweenness_centrality)
        return average_betweenness_centrality

    def cal_s_metric(G):
        return nx.s_metric(G)

    def cal_effective_size(G):
        effective_size = nx.effective_size(G)
        average_effective_size = sum(effective_size.values()) / len(effective_size)
        return average_effective_size

In [5]:
trace_1 = "afm-shear/afm/10-1"
trace_2 = "afm-shear/afm/25-1"
trace_3 = "afm-shear/afm/25-2"
trace_4 = "afm-shear/afm/50-2"

In [6]:
result_2 = []
for trace in [trace_1, trace_2, trace_3, trace_4]:
    lmp_xyz_out = trace + "/dump.1eqrun.xyz"
    print(f"--{lmp_xyz_out}")
    simulation_trace = xyz_traj_handle(xyz_file_path = lmp_xyz_out,
                                       timestep = 1)
    simulation_trace.extract_frames()
    frames = simulation_trace.frames // simulation_trace.timestep
    frames_interval = {41:2, 21:2, 11:1, 5:1, 401:20}
    result_3 = []
    for time in range(0, frames, frames_interval[frames]):
        pairs = find_pairs_kdtree(simulation_trace.frame_arr[time],
                                  min_distance = 0,
                                  max_distance = 1)
        G = nx.Graph()
        G.add_nodes_from(simulation_trace.frame_arr[time,:,0])
        G.add_edges_from(pairs)

        # calculate here

        count, max_size,r_total, max_total = network_analysis_function.count_subgraphs(G)
        average_degree = network_analysis_function.cal_average_degree(G)
        result_3.append([max_total, average_degree])
    result_2.append(result_3)
result_2 = np.array(result_2)

--afm-shear/afm/10-1/dump.1eqrun.xyz
Traj: frame nums 11
--afm-shear/afm/25-1/dump.1eqrun.xyz
Traj: frame nums 11
--afm-shear/afm/25-2/dump.1eqrun.xyz
Traj: frame nums 11
--afm-shear/afm/50-2/dump.1eqrun.xyz
Traj: frame nums 11


In [9]:
np.savetxt("afm-topo/equilibrium-RGPC.txt", result_2[:,:,0].T, delimiter=" ", fmt="%.4f")
np.savetxt("afm-topo/equilibrium-CDnm.txt", result_2[:,:,1].T, delimiter=" ", fmt="%.4f")